# 지도학습
- 머신러닝으로 AI 모델링하기
    - 데이터 : 국민건강보험공단_건강검진정보
- 딥러닝으로 AI 모델링하기
    - invistico_Airline

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

: 

## 1.선형회귀(Linear Regression)
- 데이터를 잘 표현하는 최적의 직선으로 예측을 하는 방법론
- y = ax + b

In [ ]:
# 데이터 만들기(y=4x+7)
np.random.seed(2023)
x = []
y = []

for a in range(1,11):
    x.append(a)
    y.append(4*a+7)
    
#데이터 확인
print("x : ", x)
print("y : ", y)


: 

In [ ]:
# 선형회귀 라이브러리 불러오기
from sklearn.linear_model import LinearRegression

LR = LinearRegression()

# 학습을 위한 행열변경((1,10) -> (10,1))
x = np.array(x).reshape(-1,1)

# 데이터 확인하기
print('데이터 확인')
print("x : ", x)
print("x.shape : ", x.shape)

: 

In [ ]:
# 학습
LR.fit(x,y)

# coef, intercept 확인
print("기울기")
print(LR.coef_)
print("절편")
print(LR.intercept_)

# 결과 보기

# 점
plt.scatter(x, y, color = 'r', s=20)
# 선
y_hat = x * LR.coef_ + LR.intercept_
plt.plot(x, y_hat, color = 'orange')

plt.xlabel('x')
plt.ylabel('y')

plt.show()

: 

# 사례 기반 실습
- 데이터 출처 : https://www.data.go.kr/data/15007122/fileData.do
- 데이터 파일명 : 국민건강보험공단_건강검진정보

## 목표
- 다양한 데이터를 바탕으로 LDL 콜레스테롤 수치를 예측해 보는 선형회귀모델 만들기

## 가설
- 데이터 중 치아 관련 문항은 콜레스테롤과 상관없을 것이다

### 1.데이터 전처리

In [ ]:
# 데이터 불러오기
df = pd.read_csv('./지도학습_데이터/국민건강보험공단_건강검진정보_20211231.csv', encoding='cp949')

# 데이터 보는 범위 확장하기
# display 옵션을 통한 전체 열 확장
pd.set_option('display.max_columns', None)
df.head()

: 

In [ ]:
columns_name = ['기준년도', 'ID', '시도코드', '성별코드', '연령대코드(5세단위)', '신장(5Cm단위)', '체중(5Kg 단위)',
                '허리둘레', '시력(좌)', '시력(우)', '청력(좌)', '청력(우)', '수축기혈압', '이완기혈압', '공복혈당', 
                '총 콜레스테롤', '트리글리세라이드', 'HDL콜레스테롤', 'LDL콜레스테롤', '혈색소', '요단백', '혈청크레아티닌',
                '(혈청지오티)AST', '(혈청지오티)ALT', '감마지티피', '흡연상태', '음주여부', '구강검진 수검여부', 
                '치아우식증유무', '결손치 유무', '치아마모증유무', '사랑니 이상', '치석', '데이터 공개일차']

# 컬럼명 바꾸기
df.columns = columns_name
df.head()

: 

##### 시력, 청력, 차아 관련 칼럼은 관계없다는 가정하에 열 제거

In [ ]:
irrelevant_columns = ['시력(좌)', '시력(우)', '청력(좌)', '청력(우)', '구강검진 수검여부', '치아우식증유무', '결손치 유무', '치아마모증유무', '사랑니 이상', '치석', '데이터 공개일차']

df.drop(columns=irrelevant_columns, axis=1, inplace=True)
df.head()

: 

##### 각 칼럼 확인해서 삭제 여부 판단하기

In [ ]:
for name in df.columns:
    print(name)
    print(df[f'{name}'].value_counts())
    print('-----------------------------------------')

: 

##### 기준년도는 모든 행이 동일, ID는 모두 1개씩 부여되어 있음 --> 삭제하기

##### 시도코드, 성별코드는 가설을 확인하려고 할 때, 관련없음 --> 삭제하기

In [ ]:
df.drop(columns=['ID', '기준년도', '시도코드' ,'성별코드'], axis=1, inplace=True)
df.head()

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 